# (Exploration of the Ford GoBike System Data)
## by (Dmitrij Burlak)



## Preliminary Wrangling

> Briefly introduce your dataset here.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

In [2]:
import os, zipfile

<span style="color:red">ONLY FOLLOW NEXT STEPS IF DATA IS NOT AVAILABLE AND NEEDS TO BE DOWNLOADED</span>

Download data (the .zip files) from [here](https://stackoverflow.com/questions/31346790/unzip-all-zipped-files-in-a-folder-to-that-same-folder-using-python-2-7-5) and place it in the folder 

> /data/bike/

In [3]:
# extract all data from zip files if it is not yet been extracted

for f in os.listdir('.'):
    if f.endswith('.zip'): # check for ".zip" extension
        zip_ref = zipfile.ZipFile(f) # create zipfile object
        zip_ref.extractall('.') # extract file to dir
        zip_ref.close() # close file

In [7]:
#read in all available .csv files

#This code is taken from https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
lbikesPath = 'data/bike/'
allFiles = os.listdir(lbikesPath)
dfs = []
for f in allFiles:
    print(f)
    if f.endswith('.csv'):
        df = pd.read_csv(lbikesPath + f,index_col=None, header=0);
        dfs.append(df)

Wall time: 7.11 s


In [8]:
#conncat all the dfs in one and store it
frame = pd.concat(dfs, axis = 0, ignore_index = True)
frame.to_csv('data/allData.csv');

<span style="color:red">FROM HERE IF (allData.csv exists already) YOU HAVE ALREADY THE DATA DOWNLOADED AND EXECUTED THE STEPS ABOVE ONCE.</span>

In [10]:
%%time
go_bike = pd.read_csv('data/allData.csv');

Wall time: 7.54 s


In [11]:
%%time
go_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1533136 entries, 0 to 1533135
Data columns (total 17 columns):
Unnamed: 0                 1533136 non-null int64
duration_sec               1533136 non-null int64
start_time                 1533136 non-null object
end_time                   1533136 non-null object
start_station_id           1524741 non-null float64
start_station_name         1524741 non-null object
start_station_latitude     1533136 non-null float64
start_station_longitude    1533136 non-null float64
end_station_id             1524741 non-null float64
end_station_name           1524741 non-null object
end_station_latitude       1533136 non-null float64
end_station_longitude      1533136 non-null float64
bike_id                    1533136 non-null int64
user_type                  1533136 non-null object
member_birth_year          1439858 non-null float64
member_gender              1440170 non-null object
bike_share_for_all_trip    1533136 non-null object
dtypes: float64(

In [13]:
print(go_bike.shape)
print(go_bike.dtypes)

(1533136, 17)
Unnamed: 0                   int64
duration_sec                 int64
start_time                  object
end_time                    object
start_station_id           float64
start_station_name          object
start_station_latitude     float64
start_station_longitude    float64
end_station_id             float64
end_station_name            object
end_station_latitude       float64
end_station_longitude      float64
bike_id                      int64
user_type                   object
member_birth_year          float64
member_gender               object
bike_share_for_all_trip     object
dtype: object
Wall time: 1.99 ms


In [15]:
go_bike.head()

,Unnamed: 0,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,0,75284,2018-01-31 22:52:35.2390,2018-02-01 19:47:19.8240,120.0,Mission Dolores Park,37.761420,-122.426435,285.0,Webster St at O'Farrell St,37.783521,-122.431158,2765,Subscriber,1986.0,Male,No
1,1,85422,2018-01-31 16:13:34.3510,2018-02-01 15:57:17.3100,15.0,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,15.0,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,2815,Customer,NaN,NaN,No
2,2,71576,2018-01-31 14:23:55.8890,2018-02-01 10:16:52.1160,304.0,Jackson St at 5th St,37.348759,-121.894798,296.0,5th St at Virginia St,37.325998,-121.877120,3039,Customer,1996.0,Male,No
3,3,61076,2018-01-31 14:53:23.5620,2018-02-01 07:51:20.5000,75.0,Market St at Franklin St,37.773793,-122.421239,47.0,4th St at Harrison St,37.780955,-122.399749,321,Customer,NaN,NaN,No
4,4,39966,2018-01-31 19:52:24.6670,2018-02-01 06:58:31.0530,74.0,Laguna St at Hayes St,37.776435,-122.426244,19.0,Post St at Kearny St,37.788975,-122.403452,617,Subscriber,1991.0,Male,No


## Data Assessing
### Cleaning
### Tidiness 

## Cleaning

### Define

### Code

### Test


### What is the structure of your dataset?

> Your answer here!

### What is/are the main feature(s) of interest in your dataset?

> Your answer here!

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Your answer here!

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!